# Atividade 02 - melhorar o desempenho de RP em conjunto de dados existentes

A atividade 02 visa trabalhar com um conjunto de dados pré-construído, onde as opções que o desenvolvedor tem, são de aplicar as técnicas de pré-processamento abaixo relacionadas:


Base de dados escolhida: Secondary Mushroom
Link: https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset

Sobre a base de dados:

Esta base de dados foi criada com a finalidade de simular e fornecer um conjunto de dados maior e mais variado de cogumelos hipotéticos para tarefas de aprendizado de máquina e análise de dados, inspirado não Conjunto de Dados de Cogumelos original de J. Schlimmer.

url:https://archive.ics.uci.edu/ml/datasets/Mushroom.

Este conjunto de dados inclui 61069 cogumelos hipotéticos com Píleo (chapéus do cogumelo) baseado em 173 espécies (353 cogumelos por espécie). Cada cogumelo é identificado como definitivamente comestível, definitivamente venenoso ou com classificação duvidosa e não recomendado para consumo (esta última classe combinada com a classe de cogumelos venenosos).

Descrição dos dados:

| Variável             | Papel     | Tipo       | Descrição                         | Unidade | Valores ausentes |
|----------------------|-----------|------------|-----------------------------------|---------|------------------|
| class                | Classe    | Categórico | Target (p venenoso, e comestível) | -       | não              |
| cap-diameter         | Atributo  | Contínuo   | diâmetro do píleo                 | cm      | não              |
| cap-shape            | Atributo  | Categórico | formato do píleo                  | -       | não              |
| cap-surface          | Atributo  | Categórico | superfície do píleo               | -       | sim              |
| cap-color            | Atributo  | Categórico | cor do píleo                      | -       | não              |
| does-bruise-or-bleed | Atributo  | Categórico | cogumelo machuca ou sangra        | -       | não              |
| gill-attachment      | Atributo  | Categórico | posição da lamela                 | -       | sim              |
| gill-spacing         | Atributo  | Categórico | espaçamento da lamela             | -       | sim              |
| gill-color           | Atributo  | Categórico | cor da lamela                     | -       | não              |
| stem-height          | Atributo | Contínuo   | altura da haste                   | cm      | não              |
| stem-width           | Atributo | Categórico | largura da haste                  | mm      | não              |
| stem-root            | Atributo | Categórico | tipo de raíz                      | -       | sim              |
| stem-surface         | Atributo | Categórico | superfície da haste               | -       | sim              |
| stem-color           | Atributo | Categórico | cor da haste                      | -       | não              |
| veil-type            | Atributo | Categórico | tipo de véu                       | -       | sim              |
| veil-color           | Atributo | Categórico | cor do véu                        | -       | sim              |
| has-ring             | Atributo | Categórico | possuie anel                      | -       | não              |
| ring-type            | Atributo | Categórico | tipo de anel                      | -       | sim              |
| spore-print-color    | Atributo | Categórico | cor da marca de esporos           | -       | sim              |
| habitat              | Atributo | Categórico | habitat (onde cresce)             | -       | não              |
| season               | Atributo | Categórico | estação do ano                    | -       | não              |

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
secondary_mushroom = fetch_ucirepo(id=848)

# data (as pandas dataframes)
X = secondary_mushroom.data.features
y = secondary_mushroom.data.targets

print(secondary_mushroom.data.original.head())

  class  cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
0     p         15.26         x           g         o                    f   
1     p         16.60         x           g         o                    f   
2     p         14.07         x           g         o                    f   
3     p         14.17         f           h         e                    f   
4     p         14.64         x           h         o                    f   

  gill-attachment gill-spacing gill-color  stem-height  ...  stem-root  \
0               e          NaN          w        16.95  ...          s   
1               e          NaN          w        17.99  ...          s   
2               e          NaN          w        17.80  ...          s   
3               e          NaN          w        15.77  ...          s   
4               e          NaN          w        16.53  ...          s   

  stem-surface stem-color veil-type veil-color has-ring ring-type  \
0            y   

In [3]:
len(secondary_mushroom.data.original)
# Vai precisar:
# codificação
# limpeza ?
# normalização maybe
# coreeção de prevaência, 27k de comestivel contra 33 de venenoso
y.value_counts()

class
p        33888
e        27181
Name: count, dtype: int64

## Considerações sobre a característica dos dados:

* A grande maioria dos atributos são do tipo nominal. Logo será necessário aplicar a técnica de codificação para usar em algorítmos de ML tais como o SVM.
* Alguns atributos possuem valores faltantes em uma proporção significativa. Nos casos em que a quantidade excede os 40%, adotaremos uma estratégia para eliminar a feature do modelo. Segue abaixo a taxa de proporção de dados faltantes para os atributos marcados como portadores de NaN.

In [4]:
total_samples = X.shape[0]
attributes_with_missing_values = ['cap-surface', 'gill-attachment', 'gill-spacing', 'stem-root', 'stem-surface', 'veil-type', 'veil-color', 'ring-type', 'spore-print-color']
for attribute in attributes_with_missing_values:
    print(f"Attribute: {attribute} has {(X[attribute].isna().sum() / total_samples)*100:.2f}% missing values")

Attribute: cap-surface has 23.12% missing values
Attribute: gill-attachment has 16.18% missing values
Attribute: gill-spacing has 41.04% missing values
Attribute: stem-root has 84.39% missing values
Attribute: stem-surface has 62.43% missing values
Attribute: veil-type has 94.80% missing values
Attribute: veil-color has 87.86% missing values
Attribute: ring-type has 4.05% missing values
Attribute: spore-print-color has 89.60% missing values


### Estratégia 1 - Eliminar atributos com taxa elevada de dados faltantes

Neste caso, optamos por eliminar os seguintes atributos do modelo por entender que estes agregam pouca informação:
1. veil-type
2. veil-color
3. spore-print-color
4. stem-root
5. stem-surface
6. gill-spacing

Como todos eles são nominais, fica um tanto complexo substituir por valores como média e mediana. Substituir estes pela moda ou valores aleatórios também não parece fazer sentido para estes atributos.

In [6]:
columns_to_drop = ['veil-type', 'veil-color', 'spore-print-color', 'stem-root', 'stem-surface', 'gill-spacing']
cleaned_df = X.drop(columns=columns_to_drop)

## Estratégia 2 - Eliminar linhas com atributos faltantes

Como o atributo ring-type possui dados faltantes em < 5% dos amostras, podemos eliminar as linhas que possuem esta coluna incompleta do modelo

In [11]:
lines_to_drop = cleaned_df[cleaned_df['ring-type'].isna()].index
cleaned_df = cleaned_df.drop(lines_to_drop, axis=0)

## Estratégia 3 - Fazer inputação de dados faltantes entre (15% e 25%) de informações faltantes

Agora que os atributos com excesso de dados faltantes foram eliminados e os registros com atributos ring-type faltantes foram eliminados, optamos por fazer a inputação de dados nos atributos cap-surface e gill-attachment, com 23% e 16% de valores ausentes respectivamente. Optamos por preencher os dados faltantes de cada atributo pela moda de cada um deles.

In [18]:
mode_cap_surface = cleaned_df['cap-surface'].mode()
mode_gill_attachment = cleaned_df['gill-attachment'].mode()
print(f"mode_cap_surface is unique? {len(mode_cap_surface) == 1}. Values is(are): {mode_cap_surface}")
print(f"mode_gill_attachment is unique? {len(mode_gill_attachment) == 1}. Values is(are): {mode_gill_attachment}")

mode_cap_surface is unique? True. Values is(are): 0    t
Name: cap-surface, dtype: object
mode_gill_attachment is unique? True. Valus is(are): 0    a
Name: gill-attachment, dtype: object
